# 线性回归的 Gluon 实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节中更简洁的代码来实现同样的模型。本节中，我们将介绍如何使用 MXNet 提供的 Gluon 接口更方便地实现线性回归的训练。

## 生成数据集

我们生成与上一节中相同的数据集。其中`features`是训练数据特征，`labels`是标签。

In [1]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

## 读取数据

Gluon 提供了`data`模块来读取数据。由于`data`常用作变量名，我们将导入的`data`模块用添加了 Gluon 首字母的假名`gdata`代替。在每一次迭代中，我们将随机读取包含 10 个数据样本的小批量。

In [2]:
from mxnet.gluon import data as gdata

batch_size = 10
# 将训练数据的特征和标签组合。
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量。
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

这里`data_iter`的使用跟上一节中的一样。让我们读取并打印第一个小批量数据样本。

In [3]:
for X, y in data_iter:
    print(X, y)
    break


[[ -5.42003720e-04  -4.72701073e-01]
 [  1.74664640e+00   4.39285845e-01]
 [  2.79939681e-01  -2.17812824e+00]
 [  6.50739193e-01  -1.14641559e+00]
 [  5.36511958e-01   5.02119362e-01]
 [ -1.73355961e+00   1.74691153e+00]
 [ -5.46343863e-01   1.49244428e+00]
 [ -6.45869493e-01  -8.86404574e-01]
 [ -6.36486828e-01   8.37802768e-01]
 [ -2.95661032e-01  -3.67180735e-01]]
<NDArray 10x2 @cpu(0)> 
[  5.80794668   6.18776655  12.16519737   9.40812778   3.56629658
  -5.20923519  -1.9613179    5.92847013   0.08166366   4.8691144 ]
<NDArray 10 @cpu(0)>


## 定义模型

在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤将变得更加繁琐。其实，Gluon 提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用 Gluon 更简洁地定义线性回归。

首先，导入`nn`模块。实际上，“nn”是 neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。我们先定义一个模型变量`net`，它是一个 Sequential 实例。在 Gluon 中，Sequential 实例可以看作是一个串联各个层的容器。在构造模型时，我们在该容器中依次添加层。当给定输入数据时，容器中的每一层将依次计算并将输出作为下一层的输入。

In [4]:
from mxnet.gluon import nn

net = nn.Sequential()

回顾图 3.1 中线性回归在神经网络图中的表示。作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层。在 Gluon 中，全连接层是一个`Dense`实例。我们定义该层输出个数为 1。

In [5]:
net.add(nn.Dense(1))

值得一提的是，在 Gluon 中我们无需指定每一层输入的形状，例如线性回归的输入个数。当模型看见数据时，例如后面执行`net(X)`时，模型将自动推断出每一层的输入个数。我们将在之后“深度学习计算”一章详细介绍这个机制。Gluon 的这一设计为模型开发带来便利。


## 初始化模型参数

在使用`net`前，我们需要初始化模型参数，例如线性回归模型中的权重和偏差。我们从 MXNet 导入`initializer`模块。该模块提供了模型参数初始化的各种方法。这里的`init`是`initializer`的缩写形式。我们通过`init.Normal(sigma=0.01)`指定权重参数每个元素将在初始化时随机采样于均值为 0 标准差为 0.01 的正态分布。偏差参数默认会初始化为零。

In [6]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

## 定义损失函数

在 Gluon 中，`loss`模块定义了各种损失函数。我们用假名`gloss`代替导入的`loss`模块，并直接使用它所提供的平方损失作为模型的损失函数。

In [7]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()  # 平方损失又称 L2 范数损失。

## 定义优化算法

同样，我们也无需实现小批量随机梯度下降。在导入 Gluon 后，我们创建一个`Trainer`实例，并指定学习率为 0.03 的小批量随机梯度下降（`sgd`）为优化算法。该优化算法将用来迭代`net`实例所有通过`add`函数嵌套的层所包含的全部参数。这些参数可以通过`collect_params`函数获取。

In [8]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.003})

## 训练模型

在使用 Gluon 训练模型时，我们通过调用`Trainer`实例的`step`函数来迭代模型参数。上一节中我们提到，由于变量`l`是长度为`batch_size`的一维 NDArray，执行`l.backward()`等价于执行`l.sum().backward()`。按照小批量随机梯度下降的定义，我们在`step`函数中指明批量大小，从而对批量中样本梯度求平均。

In [9]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(1)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))
    if l.mean().asnumpy() < 0.00005:
        break

epoch 1, loss: 0.035050
epoch 2, loss: 0.000122
epoch 3, loss: 0.000049


下面我们分别比较学到的和真实的模型参数。我们从`net`获得需要的层，并访问其权重（`weight`）和偏差（`bias`）。学到的和真实的参数很接近。

In [10]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.99967241 -3.40057826]]
 <NDArray 1x2 @cpu(0)>)

In [13]:
true_b, dense.bias.data()

(4.2, 
 [ 4.19966698]
 <NDArray 1 @cpu(0)>)

In [19]:
dense.weight.grad()


[[-0.00578294 -0.01616663]]
<NDArray 1x2 @cpu(0)>

In [20]:
help(gluon.loss)

Help on module mxnet.gluon.loss in mxnet.gluon:

NAME
    mxnet.gluon.loss - losses for training neural networks

CLASSES
    mxnet.gluon.block.HybridBlock(mxnet.gluon.block.Block)
        Loss
            CTCLoss
            CosineEmbeddingLoss
            HingeLoss
            HuberLoss
            KLDivLoss
            L1Loss
            L2Loss
            LogisticLoss
            PoissonNLLLoss
            SigmoidBinaryCrossEntropyLoss
            SoftmaxCrossEntropyLoss
            SquaredHingeLoss
            TripletLoss
    
    class CTCLoss(Loss)
     |  Connectionist Temporal Classification Loss.
     |  
     |  
     |  Parameters
     |  ----------
     |  layout : str, default 'NTC'
     |      Layout of prediction tensor. 'N', 'T', 'C' stands for batch size,
     |      sequence length, and alphabet_size respectively.
     |  label_layout : str, default 'NT'
     |      Layout of the labels. 'N', 'T' stands for batch size, and sequence
     |      length respectively.
  

In [22]:
help(init)

Help on module mxnet.initializer in mxnet:

NAME
    mxnet.initializer - Weight initializer.

CLASSES
    builtins.object
        Initializer
            Bilinear
            Constant
            FusedRNN
            LSTMBias
            Normal
            One
            Orthogonal
            Uniform
            Xavier
                MSRAPrelu
            Zero
        Load
        Mixed
    builtins.str(builtins.object)
        InitDesc
    
    class Bilinear(Initializer)
     |  Initialize weight for upsampling layers.
     |  
     |  Method resolution order:
     |      Bilinear
     |      Initializer
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from Initializer:
     |  
     |  __call__(self, desc, arr)
     |      Initialize an array
  

In [23]:
help(init.Xavier)

Help on class Xavier in module mxnet.initializer:

class Xavier(Initializer)
 |  Returns an initializer performing "Xavier" initialization for weights.
 |  
 |  This initializer is designed to keep the scale of gradients roughly the same
 |  in all layers.
 |  
 |  By default, `rnd_type` is ``'uniform'`` and `factor_type` is ``'avg'``,
 |  the initializer fills the weights with random numbers in the range
 |  of :math:`[-c, c]`, where :math:`c = \sqrt{\frac{3.}{0.5 * (n_{in} + n_{out})}}`.
 |  :math:`n_{in}` is the number of neurons feeding into weights, and :math:`n_{out}` is
 |  the number of neurons the result is fed to.
 |  
 |  If `rnd_type` is ``'uniform'`` and `factor_type` is ``'in'``,
 |  the :math:`c = \sqrt{\frac{3.}{n_{in}}}`.
 |  Similarly when `factor_type` is ``'out'``, the :math:`c = \sqrt{\frac{3.}{n_{out}}}`.
 |  
 |  If `rnd_type` is ``'gaussian'`` and `factor_type` is ``'avg'``,
 |  the initializer fills the weights with numbers from normal distribution with
 |  a s

## 小结

* 使用 Gluon 可以更简洁地实现模型。
* 在 Gluon 中，`data`模块提供了有关数据处理的工具，`nn`模块定义了大量神经网络的层，`loss`模块定义了各种损失函数。
* MXNet 的`initializer`模块提供了模型参数初始化的各种方法。


## 练习

* 如果将`l = loss(net(X), y)`替换成`l = loss(net(X), y).mean()`，我们需要将`trainer.step(batch_size)`相应地改成`trainer.step(1)`。这是为什么呢？
> l为小批量的损失，trainer.step(batch_size)在更新权重w的时候步幅为learing_rate/batch_size.l为小批量的损失，trainer.step(batch_size)在更新权重w的时候步幅为learing_rate/batch_size.
如果先求过均值了，那l就是小批量的平均损失, 更新权重的时候就不用再除以batch_size了。
可以试着把learing_rate改小比如0.003，然后迭代次数改大点num_epochs = 10.
对比trainer.step(1)和trainer.step(batch_size), 可以明显观察到前者收敛的更快，因为步幅更大。
如果先求过均值了，那l就是小批量的平均损失, 更新权重的时候就不用再除以batch_size了。
可以试着把learing_rate改小比如0.003，然后迭代次数改大点num_epochs = 10.
对比trainer.step(1)和trainer.step(batch_size), 可以明显观察到前者收敛的更快，因为步幅更大。
* 查阅 MXNet 文档，看看`gluon.loss`和`init`模块里提供了哪些损失函数和初始化方法。
> help(gluon.loss)         
help(init)
* 如何访问`dense.weight`的梯度？
> dense.weight.grad()


## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/742)

![](../img/qr_linear-regression-gluon.svg)